https://huggingface.co/blog/how-to-generate

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_name = "sdadas/polish-gpt2-medium"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
tokenizer.vocab_size

51200

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=False)
)

In [ ]:
try:
  dataset = pd.read_csv("data/annotations_all_batches - SENTENCE.csv")
except:
  dataset = pd.read_csv("/content/drive/MyDrive/studia/NLP/NLP-Zad2/data/annotations_all_batches - SENTENCE.csv")
dataset

,text,Olek,Kuba,Zgodne?,Stachu,Finalna anotacja
0,"""Generalnie nie mam żadnych zastrzeżeń i jak n...",2,2,T,NaN,2
1,Etui na pierwszy rzut oka wydaje się bardzo ła...,0,0,T,NaN,0
2,"Statyw fajnie wykonany, wysoki i ku mojemu zdz...",1,1,T,NaN,1
3,"przepraszam ze tak dlugo nie odpowiadalem ,ale...",2,2,T,NaN,2
4,"Brak opakowania, instrukcji przez co występują...",0,1,N,0.0,0
...,...,...,...,...,...,...
133,Szkło zle przylega do telefonu. Tylko cieniutk...,0,0,T,NaN,0
134,Sprzedającego ten produkt omijać z daleka. Spr...,0,0,T,NaN,0
135,Bardzo kiepskie wykonanie nie polecam zakupu. ...,0,0,T,NaN,0
136,"""Do codziennego użytkowania są rewelacyjne. Od...",2,2,T,NaN,2


In [ ]:
pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.9 MB/s eta 0:00:00


In [ ]:
!python -m spacy download pl_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 29.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from transformers import pipeline
import spacy
import random
import re

fill_mask = pipeline("fill-mask", model="allegro/herbert-base-cased", device=0)
nlp = spacy.load("pl_core_news_sm")

def clean_sentence(sentence):
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence = re.sub(r'\s([?.!,;:])', r'\1', sentence)
    return sentence.strip()

def mask_by_pos(sentence, pos_tags=["NOUN", "VERB", "ADJ"]):
    doc = nlp(sentence)
    candidates = [token.text for token in doc if token.pos_ in pos_tags]
    if not candidates:
        return None

    word_to_mask = random.choice(candidates)
    masked_sentence = sentence.replace(word_to_mask, "<mask>", 1)
    return masked_sentence

def generate_augmented_sentences(sentence, num_sentences=5, pos_tags=["NOUN", "VERB", "ADJ"], top_k=3):
    augmented_sentences = set()
    attempts = 0
    max_attempts = num_sentences * 2  # Limit prób, aby uniknąć nieskończonej pętli

    while len(augmented_sentences) < num_sentences and attempts < max_attempts:
        attempts += 1

        masked_sentence = mask_by_pos(sentence, pos_tags)
        if not masked_sentence:
            continue

        predictions = fill_mask(masked_sentence)
        selected_prediction = random.choice(predictions[:top_k])['sequence']
        clean_augmented_sentence = clean_sentence(selected_prediction)

        if len(clean_augmented_sentence.split()) >= len(sentence.split()) // 2:
            augmented_sentences.add(clean_augmented_sentence)

    return list(augmented_sentences)

sentence = "Generalnie nie mam żadnych zastrzeżeń i jak najbardziej polecam - pokrowiec jest solidny, świetnie wygląda (chociaż wolałbym, gdyby był matowy, ale to inna kwestia), a także sztywno trzyma pada. Jedyny, drobny minus to zamek błyskawiczny, który przy otwarciu w dziwny sposób się rozkłada, ale można to samemu sobie poprawić, poza tym nie wpływa to na nic."

augmented_sentences = generate_augmented_sentences(
    sentence,
    num_sentences=5,
    pos_tags=["NOUN", "VERB", "ADJ"],  # Maskowanie rzeczowników, czasowników, przymiotników
    top_k=3
)

# Wyświetl wyniki
print("Oryginalne zdanie:", sentence)
print("\nNowe zdania:")
for i, new_sentence in enumerate(augmented_sentences, 1):
    print(f"{i}. {new_sentence}")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Oryginalne zdanie: Generalnie nie mam żadnych zastrzeżeń i jak najbardziej polecam - pokrowiec jest solidny, świetnie wygląda (chociaż wolałbym, gdyby był matowy, ale to inna kwestia), a także sztywno trzyma pada. Jedyny, drobny minus to zamek błyskawiczny, który przy otwarciu w dziwny sposób się rozkłada, ale można to samemu sobie poprawić, poza tym nie wpływa to na nic.

Nowe zdania:
1. Generalnie nie mam żadnych zastrzeżeń i jak najbardziej polecam - pokrowiec jest solidny, świetnie wygląda ( chociaż wolałbym, gdyby był matowy, ale to inna kwestia ), a także sztywno trzyma pada. Jedyny, drobny minus to zamek błyskawiczny, który przy otwarciu w dziwny sposób się rozkłada, ale można to samemu sobie poprawić, poza tym nie wpływa to na nic.
2. Generalnie nie mam żadnych zastrzeżeń i jak najbardziej polecam - pokrowiec jest solidny, świetnie wygląda ( chociaż wolałbym, gdyby był matowy, ale to inna kwestia ), a także sztywno trzyma pada. Jedyny, drobny minus to zamek błyskawiczny, który 

In [ ]:
def augment_dataframe(df, text_column, label_column, num_augmentations=2, top_k=2, pos_tags=["NOUN", "VERB", "ADJ"]):
    augmented_data = []

    for _, row in df.iterrows():
        original_text = row[text_column]
        label = row[label_column]

        new_texts = generate_augmented_sentences(
            original_text,
            num_sentences=num_augmentations,
            pos_tags=pos_tags,
            top_k=top_k
        )

        augmented_data.append((original_text, label))
        for new_text in new_texts:
            augmented_data.append((new_text, label))

    augmented_df = pd.DataFrame(augmented_data, columns=[text_column, label_column])
    return augmented_df

In [ ]:
bert_mask_data_aug = augment_dataframe(dataset, "text", "Finalna anotacja",  num_augmentations=5)
bert_mask_data_aug.to_csv("/content/drive/MyDrive/studia/NLP/NLP-Zad2/data/annotations_all_batches - SENTENCE_AUGMENTED_BERT.csv", index=False)